In [ ]:
import ipyparallel as ipp
cluster = ipp.Client()

In [ ]:
v = cluster[:]
lview = cluster.load_balanced_view()
len(v)

 # Import necessary liblaries

In [ ]:
%%px --local

import os
try:
    import mkl
    mkl.set_num_threads(1)
except:
    pass

os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1 
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1                              #Block multithreading
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

import kwant
import numpy as np
import sympy
import sympy.physics.matrices
from scipy.constants import physical_constants
from functools import lru_cache
import tinyarray
import scipy.sparse.linalg as sla
import types
import scipy

In [ ]:
import adaptive
adaptive.notebook_extension()
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.rcParams.update({'font.size': 22})

In [ ]:
%%px --local
# Usefull constants
sigma0 = tinyarray.array([[1, 0], [0, 1]])
sigmax = tinyarray.array([[0, 1], [1, 0]])
sigmay = tinyarray.array([[0, -1j], [1j, 0]])
sigmaz = tinyarray.array([[1, 0], [0, -1]])
sigma1 = tinyarray.array([[1, 1], [1, 1]])
sigmax1 = tinyarray.array([[0, 1], [0, 0]])
sigmax2 = tinyarray.array([[0, 0], [1, 0]])
sigma1 = tinyarray.array([[1, 1], [1, 1]])
c = physical_constants['speed of light in vacuum'][0]
val_hbar = physical_constants['Planck constant over 2 pi in eV s'][0]
val_mu_B = physical_constants['Bohr magneton in eV/T'][0]
val_e = physical_constants['elementary charge'][0]
val_m0 = physical_constants['electron mass energy equivalent in MeV'][0]
val_m0 = val_m0 / (c*10**9)**2 * 10**6 # [eV * s**2 / nm**2]
val_hbarJ = 1.054571800*1e-34*1e18

In [ ]:
%%px --local

#System parameters
dx = 12
m = 0.014
par = types.SimpleNamespace(
        t=val_hbar**2/val_m0/m/dx**2/2,
        mu = 40*1e-3,
        length = 1400,
        normal_length = 600,
        normal_width = 1000,
        width = 1000,
        g = -50,    
        B = .0001,
        phi = 0,
        delta = 2*1e-3,
        alpha = 50*1e-3,
        xt = 1000,
        yt = 0,
        tip_d = 50,
        tip_potential = 0.1)

In [ ]:
%%px --local

sc_flux_quantum = 2.067833848*1e-15
L = par.normal_length*1e-9
W = par.normal_width*1e-9
B_period = sc_flux_quantum/L/W*1e3

In [ ]:
%%px --local
 # kwant system definition
def make_system(par):
    
    ### Define the system ###
    def system(pos):
        (x, y) = pos
        ret = False
        
        if (-par.width / 2 < x  < par.width / 2 and -par.length/ 2 < y < par.length/2):
            ret = True
            
        return ret
    

    ### Define the onsite and hopping elements ###    
    def onsite(site): 
        (x, y) = site.pos
        
        g_to_apply = 0
        delta_to_apply1 = par.delta
        delta_to_apply2 = par.delta
        
        if (-par.normal_length/2 < y < par.normal_length/2):        
            delta_to_apply1 = 0
            delta_to_apply2 = 0
            g_to_apply = par.g
            
        if (y >= par.normal_length/2):
            delta_to_apply1 = par.delta*np.exp(1.j*par.phi)
            delta_to_apply2 = par.delta*np.exp(-1.j*par.phi)
            
        tip_pot = par.tip_potential/\
                        (1+(x - par.xt)**2/par.tip_d**2+(y - par.yt)**2/par.tip_d**2)
        
        return (4*par.t - par.mu + tip_pot) * np.kron(sigma0, sigmaz)  \
    + 0.5 * par.B * g_to_apply * val_mu_B * np.kron(sigmaz, sigma0)\
    + delta_to_apply1*np.kron(sigma0, sigmax1)\
    + delta_to_apply2*np.kron(sigma0, sigmax2)   
        
    
    
    def hopping_x(site1, site2):
        
        (x1, y1) = site1.pos
        (x2, y2) = site2.pos     
            
        return (-par.t)*np.kron(sigma0, sigmaz)+\
                (-(1.j/2/dx*par.alpha))*np.kron(sigmay, sigmaz)
     
    def hopping_y(site1, site2):
        (x1, y1) = site1.pos
        (x2, y2) = site2.pos
       
        peiersp = (np.cos(val_e/val_hbarJ*(x1+x2)/2*(y2-y1)*par.B)* np.kron(sigma1, sigma0)\
                  + 1.j*np.sin(val_e/val_hbarJ*(x1+x2)/2*(y2-y1)*par.B) * np.kron(sigma1, sigmaz))
        
        alpha_to_apply= par.alpha
        
        if (par.normal_length/2 < y1 or y1 < -par.normal_length/2):        
            peiersp = 1

        ret = (-par.t)*np.kron(sigma0, sigmaz)*peiersp\
            + 1.j/2/dx*alpha_to_apply*np.kron(sigmax, sigmaz)*peiersp
          
        
        return ret   
    
    lat = kwant.lattice.square(dx, norbs=4)
    sys = kwant.Builder()
    sys[lat.shape(system, (0, 0))] = onsite
    sys[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    sys[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y

    return sys

In [ ]:
%%px --local
def calculate_energies(par, phase,n_values):
        """Compute eigenenergies using MUMPS as a sparse solver.

        Parameters:
        ----------
        ham : coo_matrix
            The Hamiltonian of the system in sparse representation..
        n_eigs : int
            The number of energy eigenvalues to be returned.
        n_vec_lanczos : int
            Number of Lanczos vectors used by the sparse solver.
        sigma : float
            Parameter used by the shift-inverted method. See
            documentation of scipy.sparse.linalg.eig

        Returns:
        --------
        A list containing the sorted energy levels. Only positive
        energies are returned.
        """
        class LuInv(sla.LinearOperator):
            def __init__(self, A):
                inst = kwant.linalg.mumps.MUMPSContext()
                inst.factor(A, ordering='metis')
                self.solve = inst.solve
                try:
                    super(LuInv, self).__init__(shape=A.shape, dtype=A.dtype,
                                                matvec=self._matvec)
                except TypeError:
                    super(LuInv, self).__init__(shape=A.shape, dtype=A.dtype)

            def _matvec(self, x):
                return self.solve(x.astype(self.dtype))

        n_eigs=n_values
        n_vec_lanczos=3*n_values+10
        sigma=0.0
        par.phi = phase
        sys = make_system(par).finalized()
        ham_mat = sys.hamiltonian_submatrix(sparse=True)
        ev = sla.eigs(ham_mat, k=n_eigs, OPinv=LuInv(ham_mat), sigma=sigma, ncv=n_vec_lanczos,\
                      return_eigenvectors=False)
        
        return ev[ev>0]

In [ ]:
%%px --local
def plot_spectrum(par, phase_values, n_values = 100, E_max = 1, filename = ""):

    ev = lview.map_async(lambda phase : calculate_energies(par = par, phase = phase,n_values=n_values), phase_values)
    ev.wait_interactive()
    energies = ev.get()
        
    plt.figure(figsize=(10,7))
    plt.plot(phase_values/np.pi, np.sort(energies)*1e3, 'k-')
    plt.ylim(0, E_max)
    plt.xlabel("$\phi$ [$\pi$]")
    plt.ylabel("E [meV]")
    if(filename != ""):
        plt.savefig(filename)
    plt.show()
    
    return energies

In [ ]:
%%px --local
def estimate_coherence_length(par):
    c = physical_constants['speed of light in vacuum'][0]
    val_hbar = physical_constants['Planck constant over 2 pi in eV s'][0]
    val_mu_B = physical_constants['Bohr magneton in eV/T'][0]
    val_e = physical_constants['elementary charge'][0]
    val_m0 = physical_constants['electron mass energy equivalent in MeV'][0]
    val_m0 = val_m0 / (c*10**9)**2 * 10**6 # [eV * s**2 / nm**2]
    val_hbarJ = 1.054571800*1e-34*1e18
    
    vf = np.sqrt(2*par.mu/val_m0/m)
    xi = val_hbar*vf/par.delta
    
    return xi

In [ ]:
print("Coherence length = ", estimate_coherence_length(par))

In [ ]:
sys = make_system(par).finalized()
kwant.plot(sys, site_color=lambda site: sys.hamiltonian(site, site)[0,1], fig_size=(20,10), show = False);
plt.title("Gap distribution")
plt.xlabel("x [nm]")
plt.ylabel("y [nm]")
plt.show()
kwant.plot(sys, site_color=lambda site: sys.hamiltonian(site, site)[0,0], fig_size=(20,10), show = False);
plt.title("Potential distribution")
plt.xlabel("x [nm]")
plt.ylabel("y [nm]")
plt.show()

In [ ]:
phase_values = np.linspace(0,np.pi*2,50)
energies = plot_spectrum(par, phase_values, E_max = 2, n_values = 1000)

# overwrites data
pd.DataFrame(energies ).to_pickle('energies_long600_400SC.pkl')    #to save the dataframe, df to *.pkl\


# Lets check how current changes if we take diffrent number of states

In [ ]:
energies = pd.read_pickle('energies_long600_400SC.pkl')
positive_energies = np.real(np.array(energies))
positive_energies_sorted = np.sort(positive_energies, axis = 1)

#positive_energies
phase_values = np.linspace(0,np.pi*2,50)
Ictab=[]
for N in range(0,500):
    tab=[]
    for i in positive_energies_sorted:

        tab.append(i[0:N])

    Ic= np.max( -np.sum(np.gradient(np.array(tab)/par.delta, axis = 0)/\
                                     (phase_values[-1]-phase_values[-2]), axis = 1) )  
    Ictab.append(Ic)
plt.figure(figsize=(14,8))
plt.plot(Ictab,'k-')
plt.xlabel("N_val")
plt.ylabel("Ic [$e\Delta/\hbar$]")
plt.grid()
plt.show()

# We set n_values to 600 -- 300 positive states

In [ ]:
%%px --local
# Find critical current in function of Magnetic field
def calculate_critical_current_vs_B(B,Nphi=50, n_values = 600):
    
    phase_values = list(np.linspace(0,np.pi*2,Nphi)) # diffrent phases in which we look for critical current
    energies = []
    par.B = B
    
    for phase in phase_values:

        energies.append(calculate_energies(par,phase,n_values))
    
    energies = np.array(np.real(energies))
    current = -np.sum(np.gradient(energies/par.delta, axis = 0)/\
                                 (phase_values[-1]-phase_values[-2]), axis = 1)
    
    critical_current = np.amax(current)
    
    return critical_current

In [ ]:
learner = adaptive.Learner1D(lambda B :calculate_critical_current_vs_B(B=B, Nphi=50,n_values=600), bounds=(0,15e-3))
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.01)
runner.live_info()

# overwrites data
pd.DataFrame(learner.plot().scatter.I.data ).to_pickle('FPL600_400.pkl')    #to save the dataframe, df to *.pkl

In [ ]:
data = pd.read_pickle('FPL600_400.pkl')

plt.figure(figsize=(10,4))
plt.xlabel("$B$ [T]")
plt.ylabel("$I$ [$e\Delta/\hbar$]")
plt.grid()
plt.plot(data['x'],data['y'], "k-")

a=55
s= np.argmax(data['y'][a:])+a
plt.plot(data['x'][s],data['y'][s],'rx',markersize=20 )
print("We find first FP maximum at B = "+str(data['x'][s]*1e3)+"mT, instead of "+str(1.5*B_period)+"mT (1.5 \Phi_0)" )
plt.show()

In [ ]:
%%px --local

par.B = 5.338983050847458*1e-3

In [ ]:
phase_values = np.linspace(0,np.pi*2,200)
energies = plot_spectrum(par, phase_values, E_max = 2, n_values = 600)

# overwrites data
pd.DataFrame(energies ).to_pickle('energies2.pkl')    #to save the dataframe, df to *.pkl\


In [ ]:
energies = pd.read_pickle('energies2.pkl')
positive_energies = np.real(np.array(energies))
phase_values = np.linspace(0,np.pi*2,200)

plt.figure(figsize=(14,8))
plt.plot(phase_values/np.pi, -np.sum(np.gradient(positive_energies/par.delta, axis = 0)/\
                                 (phase_values[-1]-phase_values[-2]), axis = 1), 'k-')


plt.xlabel("$\phi$ [$\pi$]")
plt.ylabel("I [$e\Delta/\hbar$]")
plt.grid()
plt.show()

s= np.argmax( -np.sum(np.gradient(positive_energies/par.delta, axis = 0)/\
                                 (phase_values[-1]-phase_values[-2]), axis = 1) )

print("Critical current is found at phi = "+ str(phase_values[s]/np.pi) +"[pi]"  )

# Supercurrent distribution

In [ ]:
# This code is an adaptation of the source code of Kwant [C. W. Groth, M. Wimmer, A. R. Akhmerov, X. Waintal, Kwant: a software package for quantum transport, New J. Phys. 16, 063065 (2014).]

def _make_figure(dpi, fig_size):
    fig = plt.figure()

    if dpi is not None:
        fig.set_dpi(dpi)
    if fig_size is not None:
        fig.set_figwidth(fig_size[0])
        fig.set_figheight(fig_size[1])
    return fig

def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx


def streamplot_y_cut(field, box, y_min = - par.normal_length /2, y_max=par.normal_length/2, cmap=None, bgcolor=None, linecolor='k',
               max_linewidth=3, min_linewidth=1, density=2/9,
               colorbar=False, file=None,
               show=True, dpi=None, fig_size=None, ax=None,
               vmax=None):
    """Draw streamlines of a flow field in Kwant style, but trimm horizontally

    Solid colored streamlines are drawn, superimposed on a color plot of
    the flow speed that may be disabled by setting `bgcolor`.  The width
    of the streamlines is proportional to the flow speed.  Lines that
    would be thinner than `min_linewidth` are blended in a perceptually
    correct way into the background color in order to create the
    illusion of arbitrarily thin lines.  (This is done because some plot
    backends like PDF do not support lines of arbitrarily thin width.)

    Internally, this routine uses matplotlib's streamplot.

    Parameters
    ----------
    field : 3d arraylike of float
        2d array of 2d vectors.
    box : 2-sequence of 2-sequences of float
        the extents of `field`: ((x0, x1), (y0, y1))
    cmap : colormap, optional
        Colormap for the background color plot.  When not set the colormap
        "kwant_red" is used by default, unless `bgcolor` is set.
    bgcolor : color definition, optional
        The solid color of the background.  Mutually exclusive with `cmap`.
    linecolor : color definition
        Color of the flow lines.
    max_linewidth : float
        Width of lines at maximum flow speed.
    min_linewidth : float
        Minimum width of lines before blending into the background color begins.
    density : float
        Number of flow lines per point of the field.  The default value
        of 2/9 is chosen to show two lines per default width of the
        interpolation bump of `~kwant.plotter.interpolate_current`.
    colorbar : bool
        Whether to show a colorbar if a colormap is used. Ignored if `ax` is
        provided.
    file : string or file object or `None`
        The output file.  If `None`, output will be shown instead.
    show : bool
        Whether ``matplotlib.pyplot.show()`` is to be called, and the output is
        to be shown immediately.  Defaults to `True`.
    dpi : float or `None`
        Number of pixels per inch.  If not set the ``matplotlib`` default is
        used.
    fig_size : tuple or `None`
        Figure size `(width, height)` in inches.  If not set, the default
        ``matplotlib`` value is used.
    ax : ``matplotlib.axes.Axes`` instance or `None`
        If `ax` is not `None`, no new figure is created, but the plot is done
        within the existing Axes `ax`. in this case, `file`, `show`, `dpi`
        and `fig_size` are ignored.
    vmax : float or `None`
        The upper saturation limit for the colormap; flows higher than
        this will saturate.  Note that there is no corresponding vmin
        option, vmin being fixed at zero.

    Returns
    -------
    fig : matplotlib figure
        A figure with the output if `ax` is not set, else None.
    """

    # Matplotlib's "density" is in units of 30 streamlines...
    density *= 1 / 30 * np.array(field.shape[:2], int)

    # Matplotlib plots images like matrices: image[y, x].  We use the opposite
    # convention: image[x, y].  Hence, it is necessary to transpose.
    field = field.transpose(1, 0, 2)

    if field.shape[-1] != 2 or field.ndim != 3:
        raise ValueError("Only 2D field can be plotted.")

    if bgcolor is None:
        if cmap is None:
            #cmap = plt.get_cmap("afmhot_r")
            cmap = kwant._colormaps.kwant_red
        cmap = plt.get_cmap(cmap)
        bgcolor = cmap(0)[:3]
    elif cmap is not None:
        raise ValueError("The parameters 'cmap' and 'bgcolor' are "
                         "mutually exclusive.")

    if ax is None:
        fig = _make_figure(dpi, fig_size)
        ax = fig.add_subplot(1, 1, 1, aspect='equal')
    else:
        fig = None

    X = np.linspace(*box[0], num=field.shape[1])
    Y = np.linspace(*box[1], num=field.shape[0])
    
    if y_min is None:
        y_min = np.amin(Y)
    if y_max is None:
        y_max = np.amax(Y)
    
    y_min_idx = find_nearest(Y, y_min)
    y_max_idx = find_nearest(Y, y_max)

    speed = np.linalg.norm(field[y_min_idx:y_max_idx,:,:], axis=-1)
    if vmax is None:
        vmax = np.max(speed) or 1

    if cmap is None:
        ax.set_axis_bgcolor(bgcolor)
    else:
        image = ax.imshow(speed, cmap=cmap,
                          interpolation='bicubic',
                          extent=[ *box[0],y_min, y_max],
                          origin='lower', vmin=0, vmax=vmax)

    linewidth = max_linewidth / vmax * speed
    color = linewidth / min_linewidth
    thin = linewidth < min_linewidth
    linewidth[thin] = min_linewidth
    color[~ thin] = 1

    line_cmap = plt.get_cmap("Greys")

    ax.streamplot(X, Y[y_min_idx:y_max_idx], field[y_min_idx:y_max_idx,:,0],\
               field[y_min_idx:y_max_idx,:,1],
                  density=density, linewidth=linewidth,
                  color=color, cmap=line_cmap, arrowsize=2, arrowstyle='->',
                  norm=matplotlib.colors.Normalize(0, 1))

    ax.set_xlim(*box[0])
    ax.set_ylim(y_min, y_max)

    if colorbar and cmap and fig is not None:
        fig.colorbar(image)

    if(show == True):
        plt.show()
    
def current_y_cut(syst, current, relwidth=0.05, **kwargs):
    """Show an interpolated current defined for the hoppings of a system.

    The system graph together with current intensities defines a "discrete"
    current density field where the current density is non-zero only on the
    straight lines that connect sites that are coupled by a hopping term.

    To make this scalar field easier to visualize and interpret at different
    length scales, it is smoothed by convoluting it with the bell-shaped bump
    function ``f(r) = max(1 - (2*r / width)**2, 0)**2``.  The bump width is
    determined by the ``relwidth`` parameter.

    This routine samples the smoothed field on a regular (square or cubic) grid
    and displays it using an enhanced variant of matplotlib's streamplot.

    This is a convenience function that is equivalent to
    ``streamplot(*interpolate_current(syst, current, relwidth), **kwargs)``.
    The longer form makes it possible to tweak additional options of
    `~kwant.plotter.interpolate_current`.

    Parameters
    ----------
    syst : `kwant.system.FiniteSystem`
        The system for which to plot the ``current``.
    current : sequence of float
        Sequence of values defining currents on each hopping of the system.
        Ordered in the same way as ``syst.graph``. This typically will be
        the result of evaluating a `~kwant.operator.Current` operator.
    relwidth : float or `None`
        Relative width of the bumps used to smooth the field, as a fraction
        of the length of the longest side of the bounding box.
    **kwargs : various
        Keyword args to be passed verbatim to `kwant.plotter.streamplot`.

    Returns
    -------
    fig : matplotlib figure
        A figure with the output if ``ax`` is not set, else None.

    See Also
    --------
    kwant.plotter.density
    """
    return streamplot_y_cut(*kwant.plotter.interpolate_current(syst, current, relwidth), **kwargs)




In [ ]:
def plot_current(par, n_values = 600, relwidth = 0.05):
    sys = make_system(par)
    sysf = sys.finalized()
    
    class LuInv(sla.LinearOperator):
        def __init__(self, A):
            inst = kwant.linalg.mumps.MUMPSContext()
            inst.factor(A, ordering='metis')
            self.solve = inst.solve
            try:
                super(LuInv, self).__init__(shape=A.shape, dtype=A.dtype,
                                            matvec=self._matvec)
            except TypeError:
                super(LuInv, self).__init__(shape=A.shape, dtype=A.dtype)

        def _matvec(self, x):
            return self.solve(x.astype(self.dtype))

    n_eigs=n_values
    n_vec_lanczos=3*n_values+10
    sigma=0.0

    sys = make_system(par).finalized()
    ham_mat = sys.hamiltonian_submatrix(sparse=True)
    ev, evecs = sla.eigs(ham_mat, k=n_eigs, OPinv=LuInv(ham_mat), sigma=sigma, ncv=n_vec_lanczos,\
                  return_eigenvectors=True)

    ev_sorting_indices = ev.argsort()

    ev = ev[ev_sorting_indices]
    evecs = evecs[:,ev_sorting_indices]
    
    
    current = 0
    
    cl_m = tinyarray.array([[1, 0, 0, 0],\
               [0, 0, 0, 0],\
               [0, 0, 1, 0],\
               [0, 0, 0, 0]])

    J = kwant.operator.Current(sys, cl_m).bind()
    
    for iv, ee in enumerate(ev):
        if (ee>0):
            psi = evecs[:,iv]


    cl_m = tinyarray.array([[0, 0, 0, 0],\
               [0, 1, 0, 0],\
               [0, 0, 0, 0],\
               [0, 0, 0, 1]])

    J = kwant.operator.Current(sys, cl_m).bind()
    
    current = 0
    
    for iv, ee in enumerate(ev):
        if (ee>0):
            psi = evecs[:,iv]


    

    current = 0
    
    
    cl_m = -tinyarray.array([[1, 0, 0, 0],\
               [0, -1, 0, 0],\
               [0, 0, 1, 0],\
               [0, 0, 0, -1]])

    J = kwant.operator.Current(sys, cl_m).bind()
    
    for iv, ee in enumerate(ev):
        if (ee>0):
            psi = evecs[:,iv]

            

            current += J(psi)
            
    current_y_cut(sys, current, relwidth = relwidth, show = False, \
                          fig_size=(18,10), colorbar = False)
    plt.xlabel("x [nm]")
    plt.ylabel("y [nm]")
    
    #plt.savefig("long_curr_map.pdf")
    plt.show()

# Plot of first fp maximum 

In [ ]:
# with values calculated before
par.B = 5.338983050847458*1e-3
par.phi = -0.5*np.pi

In [ ]:
plt.rcParams.update({'font.size': 35})

In [ ]:

plot_current(par, relwidth=0.02)

In [ ]:
plt.rcParams.update({'font.size': 22})

## Ic(B, xt)

In [ ]:
%%px --local

def calculate_SGM_map(tipxB, phase_values = list(np.linspace(0,np.pi*2,50)), n_values = 600):
    
    
    energies = []
    par.B = tipxB[1]
    par.xt = tipxB[0]
    
    for phase in phase_values:
        
        energies.append(calculate_energies(par,phase,n_values))
    
    energies = np.array(np.real(energies))
    current = -np.sum(np.gradient(energies/par.delta, axis = 0)/\
                                 (phase_values[-1]-phase_values[-2]), axis = 1)
    
    critical_current = np.amax(current)
    return critical_current

In [ ]:
learner = adaptive.Learner2D(calculate_SGM_map, bounds=[(-600,600),(-5e-3*B_period,5e-3*B_period)])
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.0008)
runner.live_info()

In [ ]:
runner.live_info()

# overwrites data
pd.DataFrame(learner.plot().image.I.data ).to_pickle('IcmapxtB.pkl')    #to save the dataframe, df to *.pkl

In [ ]:
map_ = pd.read_pickle('IcmapxtB.pkl')

fig, ax = plt.subplots(1, figsize=(16,8))
im = ax.imshow(map_, cmap='inferno', aspect='auto', extent=[ -600, 600,-5, 5])

ax.set_ylabel("$\Phi$ [$\Phi_0$]")
ax.set_xlabel("$x_{tip}$ [nm]")
cbar = plt.colorbar(im)
cbar.set_label('$I_c$ [$e\Delta/\hbar$]')
plt.rcParams.update({'font.size': 22})
for i in range(-4,5):
    x = [-600, 600]
    t = i
    y = [t,t]
    plt.plot(x, y, color="white", linewidth=1.1)
    
for i in [-500,500]:
    y = [-5, 5]
    t = i
    x = [t,t]
    plt.plot(x, y, '--w', linewidth=1.1)

#plt.savefig("SGM_MAP_long.pdf")
plt.show()

In [ ]:
%%px --local
def calc_SGM(tipxy, phase_values = list(np.linspace(0,np.pi*2,50)), n_values = 600):
    
    
    energies = []
    par.xt = tipxy[0]
    par.yt = tipxy[1]
    for phase in phase_values:
        
        energies.append(calculate_energies(par,phase,n_values))
    
    energies = np.array(np.real(energies))
    current = -np.sum(np.gradient(energies/par.delta, axis = 0)/\
                                 (phase_values[-1]-phase_values[-2]), axis = 1)
    critical_current = np.amax(current)
    
    return critical_current

In [ ]:
%%px --local

par.B = 5.338983050847458*1e-3

In [ ]:
learner = adaptive.Learner2D(calc_SGM, bounds=[(-600,600),(-300,300)])
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.0005)
runner.live_info()

In [ ]:
runner.live_info()

# overwrites data
pd.DataFrame(learner.plot().image.I.data ).to_pickle('Icsgm.pkl')    #to save the dataframe, df to *.pkl

In [ ]:
%%px --local
# to calculate I_c0
par.B = 5.338983050847458*1e-3
par.xt = 1e6
par.yt = 1e6

In [ ]:
phase_values = np.linspace(0,np.pi*2,50)
energies = plot_spectrum(par, phase_values, E_max = 2, n_values = 600)

In [ ]:
positive_energies = np.real(np.array(energies))
positive_energies_sorted = np.sort(positive_energies, axis = 1)

I_c0= np.max( -np.sum(np.gradient(np.array( positive_energies_sorted )/par.delta, axis = 0)/\
                                     (phase_values[-1]-phase_values[-2]), axis = 1) )
print(I_c0)

In [ ]:
I_c0 =   2.3371866783172646

In [ ]:
map_ = pd.read_pickle('Icsgm.pkl')

fig, ax = plt.subplots(1, figsize=(16,8))
im = ax.imshow(map_- I_c0, cmap='bwr', aspect='auto', extent=[ -600, 600,-300, 300],vmax=2.8, vmin= -2.8)
ax.set_ylabel("$y_{tip}$ [nm]")
ax.set_xlabel("$x_{tip}$ [nm]")
cbar = plt.colorbar(im)
cbar.set_label('$I_c - I_{c0}$ [$e\Delta/\hbar$]')
plt.rcParams.update({'font.size': 22})
for i in [-500,500]:
    y = [-300, 300]
    t = i
    x = [t,t]
    plt.plot(x, y, '--w', linewidth=1.1)
    
#plt.savefig("SGM_MAP.pdf")
plt.show()